# Kansas City Postal Code Clustering
Created by: Brandon Lombardino  
Created Date: 05/12/2020

## Introduction

A business wants to know which areas of Kansas City are ideal to open a coffee shop.  The owner hopes to draw in customers by building a unique relationship within the community rather than through low prices or innovative products.  The report will investigate which area in Kansas City is best suited for the business given the owners goals and expectations

## Data and Sources

The data is based on postal codes from the Kansas City area.  The postal codes are used to define regions within the Kansas City metropolitan area and have coordinates to map their locations.  The data will be used with folium maps and a Scikit-Learn cluster model to segment the region based on venue information imported from Foursquare.

## Methodology

### Import Libraries

In [285]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
import geocoder

print('Libraries imported.')

Libraries imported.


### Import CSV

In [286]:
df_loc = pd.read_csv(r'C:\Users\brand\OneDrive\Documents\Master\IBM Data Science\Final Project\kc_loc.csv')
df_loc.head()

,postal_code,latitude,longitude
0,64102,39.092001,-94.601628
1,64105,39.104837,-94.594928
2,64106,39.106210,-94.573278
3,64108,39.085612,-94.584134
4,64109,39.066060,-94.567489


### Check the Number of Postal Codes within Kansas City

In [287]:
print('The dataframe has {} postal codes.'.format(
        len(df_loc['postal_code'].unique()),
        df_loc.shape[0]
    )
)

The dataframe has 70 postal codes.


### Find the Kansas City Coordinates

In [288]:
address = 'Kansas City, MO'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Kansas City are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Kansas City are 39.100105, -94.5781416.


### Create a Map of Kansas City

In [289]:
map_kc = folium.Map(location=[latitude, longitude], zoom_start=10.4)

# add markers to map
for lat, lng, postal_code in zip(df_loc['latitude'],df_loc['longitude'], df_loc['postal_code']):
    label = '{}'.format(postal_code)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='#E42222',
        fill=True,
        fill_color='#CB9D5B',
        fill_opacity=0.7,
        parse_html=False).add_to(map_kc)  
    
map_kc

### Use Foursquare to Explore the Area

In [290]:
CLIENT_ID = 'HRH4TIEJJO2CJFGGCCKRNAKWCMGRGQDXYCJPBHAEISQTMUQ0'
CLIENT_SECRET = 'LVEZOSHHBIYLKQODSBI5OLB01UOL3XPY4JDMUHUFEK3NDLZN'
VERSION = '20200512'

### Find the Top 500 Venues within a 1,000 Meter Radius

In [291]:
LIMIT = 500 # limit of number of venues returned by Foursquare API
radius = 1000 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=HRH4TIEJJO2CJFGGCCKRNAKWCMGRGQDXYCJPBHAEISQTMUQ0&client_secret=LVEZOSHHBIYLKQODSBI5OLB01UOL3XPY4JDMUHUFEK3NDLZN&v=20200512&ll=39.100105,-94.5781416&radius=1000&limit=500'

### Extract the Venue's Category Type

In [292]:
results = requests.get(url).json()

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

### Add the Venue and Category Columns to the Postal Code Table

In [293]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues)

filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

KeyError: 'groups'

### Create Venue Related Data

In [261]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['postal_code', 
                  'latitude', 
                  'longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)
kc_venues = getNearbyVenues(names=df_loc['postal_code'],
                                   latitudes=df_loc['latitude'],
                                   longitudes=df_loc['longitude']
                                  )

64102
64105
64106
64108
64109
64101
64110
64111


KeyError: 'groups'

### Check the Structure of the Display Table

In [262]:
print(kc_venues.shape)
kc_venues.head()

(2309, 8)


,index,postal_code,latitude,longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,0,64102,39.092001,-94.601628,Voltaire,39.093935,-94.604779,American Restaurant
1,1,64102,39.092001,-94.601628,Pole Worx,39.094161,-94.604701,Dance Studio
2,2,64102,39.092001,-94.601628,Stockyards Brewing Co.,39.094241,-94.605101,Brewery
3,3,64102,39.092001,-94.601628,The Westside Local,39.093300,-94.593200,Gastropub
4,4,64102,39.092001,-94.601628,Chez Elle,39.092578,-94.593221,French Restaurant


### Group the Number of Venues by Postal Code

In [263]:
kc_venues_groupby = kc_venues.groupby('postal_code').count()

kc_venues_groupby

,index,latitude,longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
postal_code,,,,,,,
64101,37,37,37,37,37,37,37
64102,38,38,38,38,38,38,38
64105,56,56,56,56,56,56,56
64106,69,69,69,69,69,69,69
64108,100,100,100,100,100,100,100
64109,13,13,13,13,13,13,13
64110,27,27,27,27,27,27,27
64111,53,53,53,53,53,53,53
64112,83,83,83,83,83,83,83


In [264]:
under_ten = kc_venues_groupby[kc_venues_groupby["Venue"] < 10]

under_ten

,index,latitude,longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
postal_code,,,,,,,


### Drop Postal Codes that have Less than 10 Venues

In [265]:
kc_venues_1 = kc_venues[kc_venues["postal_code"] != 64114]
kc_venues_2 = kc_venues_1[kc_venues["postal_code"] != 64117]
kc_venues_3 = kc_venues_2[kc_venues["postal_code"] != 64119]
kc_venues_4 = kc_venues_3[kc_venues["postal_code"] != 64120]
kc_venues_5 = kc_venues_4[kc_venues["postal_code"] != 64123]
kc_venues_6 = kc_venues_5[kc_venues["postal_code"] != 64125]
kc_venues_7 = kc_venues_6[kc_venues["postal_code"] != 64126]
kc_venues_8 = kc_venues_7[kc_venues["postal_code"] != 64128]
kc_venues_9 = kc_venues_8[kc_venues["postal_code"] != 64129]
kc_venues_10 = kc_venues_9[kc_venues["postal_code"] != 64131]
kc_venues_11 = kc_venues_10[kc_venues["postal_code"] != 64132]
kc_venues_12 = kc_venues_11[kc_venues["postal_code"] != 64133]
kc_venues_13 = kc_venues_12[kc_venues["postal_code"] != 64134]
kc_venues_14 = kc_venues_13[kc_venues["postal_code"] != 64136]
kc_venues_15 = kc_venues_14[kc_venues["postal_code"] != 64137]
kc_venues_16 = kc_venues_15[kc_venues["postal_code"] != 64138]
kc_venues_17 = kc_venues_16[kc_venues["postal_code"] != 64139]
kc_venues_18 = kc_venues_17[kc_venues["postal_code"] != 64146]
kc_venues_19 = kc_venues_18[kc_venues["postal_code"] != 64147]
kc_venues_20 = kc_venues_19[kc_venues["postal_code"] != 64152]
kc_venues_21 = kc_venues_20[kc_venues["postal_code"] != 64153]
kc_venues_22 = kc_venues_21[kc_venues["postal_code"] != 64154]
kc_venues_23 = kc_venues_22[kc_venues["postal_code"] != 64155]
kc_venues_24 = kc_venues_23[kc_venues["postal_code"] != 64156]
kc_venues_25 = kc_venues_24[kc_venues["postal_code"] != 64157]
kc_venues_26 = kc_venues_25[kc_venues["postal_code"] != 64158]
kc_venues_27 = kc_venues_26[kc_venues["postal_code"] != 64161]
kc_venues = kc_venues_27[kc_venues["postal_code"] != 64163]
kc_venues.reset_index(inplace = True)

print(kc_venues.shape)
kc_venues

(2309, 9)


,level_0,index,postal_code,latitude,longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,0,0,64102,39.092001,-94.601628,Voltaire,39.093935,-94.604779,American Restaurant
1,1,1,64102,39.092001,-94.601628,Pole Worx,39.094161,-94.604701,Dance Studio
2,2,2,64102,39.092001,-94.601628,Stockyards Brewing Co.,39.094241,-94.605101,Brewery
3,3,3,64102,39.092001,-94.601628,The Westside Local,39.093300,-94.593200,Gastropub
4,4,4,64102,39.092001,-94.601628,Chez Elle,39.092578,-94.593221,French Restaurant
5,5,5,64102,39.092001,-94.601628,Goat Hill Coffee & Soda,39.092903,-94.593617,Coffee Shop
6,6,6,64102,39.092001,-94.601628,Los Alamos Market y Cocina,39.093153,-94.593193,Mexican Restaurant
7,7,7,64102,39.092001,-94.601628,Blue Bird Bistro,39.093026,-94.593471,Vegetarian / Vegan Restaurant
8,8,8,64102,39.092001,-94.601628,Hello Sailor,39.099774,-94.601155,Antique Shop
9,9,9,64102,39.092001,-94.601628,Westside Storey,39.092987,-94.593291,Clothing Store


### Use One Hot Encoding on the Venue Table

In [266]:
kc_onehot = pd.get_dummies(kc_venues[['Venue Category']], prefix="", prefix_sep="")
kc_onehot['postal_code'] = kc_venues['postal_code']
fixed_columns = [kc_onehot.columns[-1]] + list(kc_onehot.columns[:-1])
kc_onehot = kc_onehot[fixed_columns]

kc_onehot.head()

,postal_code,Accessories Store,Airport,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arcade,Art Gallery,Arts & Crafts Store,Athletics & Sports,Auto Dealership,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Bed & Breakfast,Beer Bar,Beer Garden,Big Box Store,Bike Rental / Bike Share,Bistro,Board Shop,Boat or Ferry,Bookstore,Border Crossing,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Churrascaria,Clothing Store,Cocktail Bar,Coffee Shop,College Soccer Field,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Dog Run,Dry Cleaner,Electronics Store,Ethiopian Restaurant,Event Space,Farm,Farmers Market,Fast Food Restaurant,Financial or Legal Service,Flea Market,Flower Shop,Fondue Restaurant,Food Court,Food Service,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Driving Range,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Health Food Store,Herbs & Spices Store,History Museum,Hobby Shop,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Theater,Intersection,Italian Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Latin American Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Office,Other Great Outdoors,Outdoor Supply Store,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pawn Shop,Pedestrian Plaza,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plaza,Poke Place,Pool,Pop-Up Shop,Print Shop,Pub,Ramen Restaurant,Recreation Center,Rental Car Location,Restaurant,Salon / Barbershop,Sandwich Place,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Social Club,South American Restaurant,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Street Food Gathering,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tapas Restaurant,Taxi,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Tiki Bar,Tourist Information Center,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Water Park,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,64102,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,64102,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,64102,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

### Group Rows by the Postal Code and by Taking the Mean of the Frequency of Occurance of Each Category

In [267]:
kc_grouped = kc_onehot.groupby('postal_code').mean().reset_index()

kc_grouped

,postal_code,Accessories Store,Airport,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arcade,Art Gallery,Arts & Crafts Store,Athletics & Sports,Auto Dealership,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Bed & Breakfast,Beer Bar,Beer Garden,Big Box Store,Bike Rental / Bike Share,Bistro,Board Shop,Boat or Ferry,Bookstore,Border Crossing,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cheese Shop,Chinese Restaurant,Chocolate Shop,Churrascaria,Clothing Store,Cocktail Bar,Coffee Shop,College Soccer Field,Comfort Food Restaurant,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Dog Run,Dry Cleaner,Electronics Store,Ethiopian Restaurant,Event Space,Farm,Farmers Market,Fast Food Restaurant,Financial or Legal Service,Flea Market,Flower Shop,Fondue Restaurant,Food Court,Food Service,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Driving Range,Gourmet Shop,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Health Food Store,Herbs & Spices Store,History Museum,Hobby Shop,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Theater,Intersection,Italian Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Latin American Restaurant,Library,Light Rail Station,Lingerie Store,Liquor Store,Lounge,Market,Mediterranean Restaurant,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music Store,Music Venue,New American Restaurant,Nightclub,Office,Other Great Outdoors,Outdoor Supply Store,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pawn Shop,Pedestrian Plaza,Performing Arts Venue,Pet Store,Pharmacy,Pizza Place,Plaza,Poke Place,Pool,Pop-Up Shop,Print Shop,Pub,Ramen Restaurant,Recreation Center,Rental Car Location,Restaurant,Salon / Barbershop,Sandwich Place,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Soccer Stadium,Social Club,South American Restaurant,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Steakhouse,Street Food Gathering,Supermarket,Supplement Shop,Sushi Restaurant,Taco Place,Tapas Restaurant,Taxi,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Tiki Bar,Tourist Information Center,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Water Park,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,64101,0.000000,0.000000,0.000000,0.000000,0.027027,0.162162,0.00,0.000000,0.027027,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.027027,0.027027,0.000000,0.000000,0.000000,0.054054,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.027027,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.027027,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.027027,0.027027,0.000000,0.000000,0.000000,0.027027,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.027027,0.00000,0.000000,0.027027,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.027027,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.027027,0.000000,0.000000,0.027027,0.00,0.000000,0.000000,0.00000,0.000000,0.054054,0.027027,0.000000,0.000000,0.00000

### Display Each Postal Code with Top 5 Most Common Venues

In [268]:
num_top_venues = 5

for code in kc_grouped['postal_code']:
    print("----"+str(code)+"----")
    temp = kc_grouped[kc_grouped['postal_code'] == code].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----64101----
             venue  freq
0     Antique Shop  0.16
1             Park  0.08
2              Gym  0.05
3  Border Crossing  0.05
4      Coffee Shop  0.03


----64102----
               venue  freq
0       Antique Shop  0.08
1        Art Gallery  0.08
2                Gym  0.05
3  French Restaurant  0.05
4               Park  0.05


----64105----
           venue  freq
0   Antique Shop  0.12
1       Dive Bar  0.05
2     Food Truck  0.04
3  Deli / Bodega  0.04
4        Theater  0.04


----64106----
                   venue  freq
0         Sandwich Place  0.06
1  Vietnamese Restaurant  0.06
2     Italian Restaurant  0.04
3               Dive Bar  0.04
4                   Park  0.04


----64108----
                     venue  freq
0      American Restaurant  0.05
1  New American Restaurant  0.05
2                  Brewery  0.04
3       Mexican Restaurant  0.04
4              Coffee Shop  0.04


----64109----
                    venue  freq
0           Deli / Bodega  0.08
1  Thrif

### Place the Top 10 Most Common Venues into a Pandas Dataframe

In [269]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['postal_code']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
postal_code_venues_sorted = pd.DataFrame(columns=columns)
postal_code_venues_sorted['postal_code'] = kc_grouped['postal_code']

for ind in np.arange(kc_grouped.shape[0]):
    postal_code_venues_sorted.iloc[ind, 1:] = return_most_common_venues(kc_grouped.iloc[ind, :], num_top_venues)

postal_code_venues_sorted.head()

,postal_code,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,64101,Antique Shop,Park,Border Crossing,Gym,Pedestrian Plaza,Southern / Soul Food Restaurant,Social Club,Building,Café,Coffee Shop
1,64102,Antique Shop,Art Gallery,Gym,French Restaurant,Park,New American Restaurant,Brewery,Salon / Barbershop,Recreation Center,Clothing Store
2,64105,Antique Shop,Dive Bar,Deli / Bodega,Gym,Theater,Hotel,Food Truck,Park,Cocktail Bar,Coffee Shop
3,64106,Vietnamese Restaurant,Sandwich Place,Park,Italian Restaurant,Coffee Shop,Pizza Place,Dive Bar,Bakery,French Restaurant,Grocery Store
4,64108,New American Restaurant,American Restaurant,Mexican Restaurant,Coffee Shop,Brewery,Hotel,Gym,Italian Restaurant,Burger Joint,Thai Restaurant


### Create Postal Code Clusters

In [270]:
# set number of clusters
kclusters = 5

kc_grouped_clustering = kc_grouped.drop('postal_code', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(kc_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

# add clustering labels
postal_code_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

df_kc = df_loc

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
df_kc = df_kc.join(postal_code_venues_sorted.set_index('postal_code'), on='postal_code')

df_kc.head() # check the last columns!

,postal_code,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,64102,39.092001,-94.601628,2.0,Antique Shop,Art Gallery,Gym,French Restaurant,Park,New American Restaurant,Brewery,Salon / Barbershop,Recreation Center,Clothing Store
1,64105,39.104837,-94.594928,2.0,Antique Shop,Dive Bar,Deli / Bodega,Gym,Theater,Hotel,Food Truck,Park,Cocktail Bar,Coffee Shop
2,64106,39.106210,-94.573278,0.0,Vietnamese Restaurant,Sandwich Place,Park,Italian Restaurant,Coffee Shop,Pizza Place,Dive Bar,Bakery,French Restaurant,Grocery Store
3,64108,39.085612,-94.584134,0.0,New American Restaurant,American Restaurant,Mexican Restaurant,Coffee Shop,Brewery,Hotel,Gym,Italian Restaurant,Burger Joint,Thai Restaurant
4,64109,39.066060,-94.567489,3.0,Thrift / Vintage Store,Park,Fast Food Restaurant,Bus Stop,Discount Store,Theater,Juice Bar,Automotive Shop,Candy Store,Southern / Soul Food Restaurant


### Check the Cluster Label Distribution

In [271]:
df_kc.fillna(-1).groupby(["Cluster Labels"]).count()

,postal_code,latitude,longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
Cluster Labels,,,,,,,,,,,,,
-1.0,33,33,33,33,33,33,33,33,33,33,33,33,33
0.0,19,19,19,19,19,19,19,19,19,19,19,19,19
1.0,1,1,1,1,1,1,1,1,1,1,1,1,1
2.0,5,5,5,5,5,5,5,5,5,5,5,5,5
3.0,11,11,11,11,11,11,11,11,11,11,11,11,11
4.0,1,1,1,1,1,1,1,1,1,1,1,1,1


### Drop NAN Clusters

In [272]:
df_kc.dropna(inplace = True)

df_kc

,postal_code,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,64102,39.092001,-94.601628,2.0,Antique Shop,Art Gallery,Gym,French Restaurant,Park,New American Restaurant,Brewery,Salon / Barbershop,Recreation Center,Clothing Store
1,64105,39.104837,-94.594928,2.0,Antique Shop,Dive Bar,Deli / Bodega,Gym,Theater,Hotel,Food Truck,Park,Cocktail Bar,Coffee Shop
2,64106,39.106210,-94.573278,0.0,Vietnamese Restaurant,Sandwich Place,Park,Italian Restaurant,Coffee Shop,Pizza Place,Dive Bar,Bakery,French Restaurant,Grocery Store
3,64108,39.085612,-94.584134,0.0,New American Restaurant,American Restaurant,Mexican Restaurant,Coffee Shop,Brewery,Hotel,Gym,Italian Restaurant,Burger Joint,Thai Restaurant
4,64109,39.066060,-94.567489,3.0,Thrift / Vintage Store,Park,Fast Food Restaurant,Bus Stop,Discount Store,Theater,Juice Bar,Automotive Shop,Candy Store,Southern / Soul Food Restaurant
5,64101,39.104106,-94.599688,2.0,Antique Shop,Park,Border Crossing,Gym,Pedestrian Plaza,Southern / Soul Food Restaurant,Social Club,Building,Café,Coffee Shop
6,64110,39.034480,-94.572904,3.0,Sandwich Place,Fried Chicken Joint,Fast Food Restaurant,Bus Stop,Food Service,Liquor Store,Museum,Bus Station,Grocery Store,Juice Bar
7,64111,39.057304,-94.593710,0.0,Coffee Shop,Burger Joint,Liquor Store,Dive Bar,Gay Bar,Smoke Shop,Print Shop,Cocktail Bar,Ice Cream Shop,Chinese Restaurant
8,64112,39.036082,-94.595006,0.0,American Restaurant,Hotel,Seafood Restaurant,Pizza Place,Cosmetics Shop,Coffee Shop,Sandwich Place,Mexican Restaurant,Spa,Jewelry Store
9,64113,39.013748,-94.595464,3.0,Grocery Store,Coffee Shop,American Restaurant,Sandwich Place,Ice Cream Shop,Mexican Restaurant,Flea Market,Shopping Plaza,Supplement Shop,Flower Shop


### Change Postal Code from a Float into a String

In [273]:
df_kc = df_kc.astype({"Cluster Labels": int})
df_kc.reset_index(inplace = True)

df_kc.head()

,index,postal_code,latitude,longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,0,64102,39.092001,-94.601628,2,Antique Shop,Art Gallery,Gym,French Restaurant,Park,New American Restaurant,Brewery,Salon / Barbershop,Recreation Center,Clothing Store
1,1,64105,39.104837,-94.594928,2,Antique Shop,Dive Bar,Deli / Bodega,Gym,Theater,Hotel,Food Truck,Park,Cocktail Bar,Coffee Shop
2,2,64106,39.106210,-94.573278,0,Vietnamese Restaurant,Sandwich Place,Park,Italian Restaurant,Coffee Shop,Pizza Place,Dive Bar,Bakery,French Restaurant,Grocery Store
3,3,64108,39.085612,-94.584134,0,New American Restaurant,American Restaurant,Mexican Restaurant,Coffee Shop,Brewery,Hotel,Gym,Italian Restaurant,Burger Joint,Thai Restaurant
4,4,64109,39.066060,-94.567489,3,Thrift / Vintage Store,Park,Fast Food Restaurant,Bus Stop,Discount Store,Theater,Juice Bar,Automotive Shop,Candy Store,Southern / Soul Food Restaurant


### Map the Clusters

In [274]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_kc['latitude'], df_kc['longitude'], df_kc['postal_code'], df_kc['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Results

### Cluster 1 - City Centre
Represented on the map in red \
Defined by different types of bars and entertainment venues \
Caters to nightlife and tourists

In [248]:
df_kc.loc[df_kc['Cluster Labels'] == 0, df_kc.columns[[1] + list(range(5, df_kc.shape[1]))]]

,postal_code,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,64106,Vietnamese Restaurant,Sandwich Place,Park,Italian Restaurant,Coffee Shop,Pizza Place,Dive Bar,Bakery,French Restaurant,Grocery Store
3,64108,New American Restaurant,American Restaurant,Mexican Restaurant,Coffee Shop,Brewery,Hotel,Gym,Italian Restaurant,Burger Joint,Thai Restaurant
7,64111,Coffee Shop,Burger Joint,Liquor Store,Dive Bar,Gay Bar,Smoke Shop,Print Shop,Cocktail Bar,Ice Cream Shop,Chinese Restaurant
8,64112,American Restaurant,Hotel,Seafood Restaurant,Pizza Place,Cosmetics Shop,Coffee Shop,Sandwich Place,Mexican Restaurant,Spa,Jewelry Store
16,64141,Theater,Hotel,Coffee Shop,American Restaurant,Lounge,Brewery,Music Venue,Sports Bar,Dive Bar,Beer Bar
17,64144,American Restaurant,Brewery,History Museum,Mexican Restaurant,Coffee Shop,Park,Gym,Burger Joint,Hotel,Museum
18,64148,Theater,Hotel,Coffee Shop,American Restaurant,Lounge,Brewery,Music Venue,Sports Bar,Dive Bar,Beer Bar
21,64170,Theater,Hotel,Coffee Shop,American Restaurant,Lounge,Brewery,Music Venue,Sports Bar,Dive Bar,Beer Bar
23,64187,Theater,Hotel,Coffee Shop,American Restaurant,Lounge,Brewery,Music Venue,Sports Bar,Dive Bar,Beer Bar
25,64197,Theater,Hotel,Coffee Shop,American Restaurant,Lounge,Brewery,Music Venue,Sports Bar,Dive Bar,Beer Bar


### Cluster 2 - Industrial Oldtown
Represented on the map in blue \
Defined by antique shops, border crossings, and parks \
Caters to the old town and blue collar community

In [249]:
df_kc.loc[df_kc['Cluster Labels'] == 2, df_kc.columns[[1] + list(range(5, df_kc.shape[1]))]]

,postal_code,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,64102,Antique Shop,Art Gallery,Gym,French Restaurant,Park,New American Restaurant,Brewery,Salon / Barbershop,Recreation Center,Clothing Store
1,64105,Antique Shop,Dive Bar,Deli / Bodega,Gym,Theater,Hotel,Food Truck,Park,Cocktail Bar,Coffee Shop
5,64101,Antique Shop,Park,Border Crossing,Gym,Pedestrian Plaza,Southern / Soul Food Restaurant,Social Club,Building,Café,Coffee Shop
22,64171,Antique Shop,Border Crossing,Park,Cocktail Bar,Art Gallery,Kitchen Supply Store,Coffee Shop,Gym / Fitness Center,Café,Electronics Store
24,64188,Antique Shop,Border Crossing,Park,Cocktail Bar,Art Gallery,Kitchen Supply Store,Coffee Shop,Gym / Fitness Center,Café,Electronics Store


### Cluster 3 - Suburbs
Represented on the map in green \
Defined by restaurants and shopping centers \
Caters to residents who value space and quiet over community and entertainment

In [250]:
df_kc.loc[df_kc['Cluster Labels'] == 3, df_kc.columns[[1] + list(range(5, df_kc.shape[1]))]]

,postal_code,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,64109,Thrift / Vintage Store,Park,Fast Food Restaurant,Bus Stop,Discount Store,Theater,Juice Bar,Automotive Shop,Candy Store,Southern / Soul Food Restaurant
6,64110,Sandwich Place,Fried Chicken Joint,Fast Food Restaurant,Bus Stop,Food Service,Liquor Store,Museum,Bus Station,Grocery Store,Juice Bar
9,64113,Grocery Store,Coffee Shop,American Restaurant,Sandwich Place,Ice Cream Shop,Mexican Restaurant,Flea Market,Shopping Plaza,Supplement Shop,Flower Shop
10,64116,Sandwich Place,Grocery Store,Fast Food Restaurant,Convenience Store,Brewery,Outdoor Supply Store,Pub,Intersection,Street Food Gathering,Discount Store
11,64127,Mexican Restaurant,Sandwich Place,Discount Store,Business Service,Fast Food Restaurant,Park,Pawn Shop,Intersection,Latin American Restaurant,Warehouse Store
12,64118,Park,Fast Food Restaurant,American Restaurant,Mexican Restaurant,Hobby Shop,Sandwich Place,Restaurant,Chinese Restaurant,Gym,Pawn Shop
14,64124,Park,Mexican Restaurant,Convenience Store,Coffee Shop,Mobile Phone Shop,Museum,Food Truck,Taco Place,Latin American Restaurant,Fried Chicken Joint
15,64130,Fast Food Restaurant,Electronics Store,Shopping Plaza,Bus Stop,Shoe Store,Discount Store,Sandwich Place,Baseball Field,Grocery Store,American Restaurant
19,64145,Sandwich Place,Big Box Store,Furniture / Home Store,Arts & Crafts Store,Automotive Shop,Fast Food Restaurant,Golf Driving Range,Discount Store,Department Store,Yoga Studio
20,64151,Coffee Shop,Video Store,Fast Food Restaurant,Breakfast Spot,Park,Restaurant,Chinese Restaurant,Thrift / Vintage Store,Pharmacy,Gym


## Discussion and Conclusion

The cluster I labeled "Industrial Oldtown" seems best suited for a community coffee shop.  The antiques and other local \
venues demonstrate a high level of community value.  The area seems to support local shops over nationwide chains \
more than the average community.  The cluster contains a constant flow of potential customers because an abundance of \
industrial jobs attracts workers who live outside of the area.  More specifically, the coffee shop should look to open \
within the postal code 64109 region because it remains the only area without a coffee shop as a top 10 common venue. \
Postal code 64109 makes up the southern portion of an area known as the West Bottoms.  The combination of local support \
and less competition make the southern portion of the West Bottoms community in Kansas City an ideal place to open a \
community and relationship driven coffee shop.